### Chargement de données

In [19]:
import pandas as pd

df = pd.read_csv('../data/processed/data_stemmed.csv')

### 1. Vectoriser le texte à l’aide de TfidfVectorizer()

- Label et label_text

In [29]:
df[["label", "label_text"]].drop_duplicates()

,label,label_text
0,1,spam
1,0,ham


- Séparer features et labels

In [20]:
y = df["label"]
X = df["stemmed_text"]

- Split train/test

In [21]:
from sklearn.model_selection import train_test_split

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("X - Train", len(X_train_text))
print("X - Test", len(X_test_text))
print("y - Train", len(y_train))
print("y - Test", len(y_test))

X - Train 23040
X - Test 5761
y - Train 23040
y - Test 5761


- Vectorisation du texte

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer


# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=8000)
X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

print(X_train.shape, X_test.shape)

(23040, 8000) (5761, 8000)


### 2. Entraîner plusieurs modèles de classification.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

# 3. Liste des modèles à tester
models = {
    "Logistic Regression": LogisticRegression(max_iter=300),
    "Linear SVM": LinearSVC(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(n_estimators=200),
    "SGDClassifier": SGDClassifier(loss="hinge"),
}

# 4. Stocker les résultats
results = []

for name, model in models.items():
    print(f"🔹 Training {name}...")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds, pos_label=1)
    recall = recall_score(y_test, preds, pos_label=1)
    f1 = f1_score(y_test, preds, pos_label=1)

    results.append([name, accuracy, precision, recall, f1])

# 5. Afficher les résultats
import pandas as pd

results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-score"])


🔹 Training Logistic Regression...
🔹 Training Linear SVM...
🔹 Training Naive Bayes...
🔹 Training Random Forest...
🔹 Training SGDClassifier (SVM-like)...


In [42]:
results_df

,Model,Accuracy,Precision,Recall,F1-score
0,Logistic Regression,0.985072,0.976915,0.991447,0.984127
1,Linear SVM,0.988196,0.984832,0.989959,0.987389
2,Naive Bayes,0.981427,0.976032,0.984381,0.980189
3,Random Forest,0.984378,0.977933,0.988843,0.983358
4,SGDClassifier,0.988544,0.983413,0.992190,0.987782
